In [ ]:
# default_exp fault_generators

# Fault generators

> Collection of generators to produce faulty circuits.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
#export
from qsam.circuit import Circuit
import numpy as np
import itertools as it

A faulty circuit is an "empty" circuit of the same size as a reference circuit which has "fault" gates at some circuit locations. Fault gates are just regular gates (X,Y,Z,..) just with the difference that these are executed in the simulation **after** the gates in a tick of the reference circuit. Thus, a fault circuit is always defined with respect to a reference circuit. Here an example:

In [ ]:
ref_circ = Circuit([{"H":{0,1,2}}, {"CNOT": {(0,1), (0,2)}}, {"measure":{0,1,2}}])
f_circ = Circuit([dict()] * 3)
f_circ[1] = {"X": {0,2}}

print(ref_circ)
print(f_circ)

0: {'H': {0, 1, 2}}
1: {'CNOT': {(0, 1), (0, 2)}}
2: {'measure': {0, 1, 2}}

0: {}
1: {'X': {0, 2}}
2: {}


The fault generator implements a function to populate the fault circuit (randomly) due to some specified parameters. For now, we only implement a generator for depolarizing circuit-level noise, i.e. equal probability of errors on each given circuit element. For now we also only consider gates as circuit elements (1-qubit, 2-qubit, measurements). In the future, we will also implement a way to place idle and cross-talk noise.

In [ ]:
#export

ONE_QUBIT_FAULTS = ["X", "Y", "Z"]
TWO_QUBIT_FAULTS = list(it.product(ONE_QUBIT_FAULTS + ["I"], repeat=2))
TWO_QUBIT_FAULTS.remove(("I","I"))

class Depolar:
    """Fault circuit generator under depolarizing circuit-level noise"""
    
    def __init__(self, n_ticks):
        self.n_ticks = n_ticks
        
    def generate(self, partitions, params, sampler_type):
        
        if sampler_type == "DirectSampler":
            faults = [fault for partition, p in zip(partitions, params) for fault in partition if np.random.random() < p ]
        elif sampler_type == "SubsetSampler":
            faults = [partition[idx] for partition, weight in zip(partitions,params) for idx in np.random.choice(len(partition),weight,replace=False)]
        else:
            raise Exception(f"Sampler type {sampler_type} not implemented")
            
        return self._place_faults(faults)
    
    def _place_faults(self, faults):
        """Place `faults` at specified `tick_index`s in empty circuit"""
        
        fault_circuit = Circuit([{} for _ in range(self.n_ticks)])
        
        for (tick_index, qubit) in faults:
            if type(qubit) == int:
                f_gate = np.random.choice(ONE_QUBIT_FAULTS)
                qb_set = fault_circuit[tick_index].get(f_gate, set())
                qb_set.add(qubit)
                fault_circuit[tick_index][f_gate] = qb_set
            elif type(qubit) == tuple:
                f_gates = TWO_QUBIT_FAULTS[np.random.choice(len(TWO_QUBIT_FAULTS))]
                for f_gate, qubit_i in zip(f_gates, qubit):
                    if f_gate != "I":
                        qb_set = fault_circuit[tick_index].get(f_gate, set())
                        qb_set.add(qubit_i)
                        fault_circuit[tick_index][f_gate] = qb_set
        return fault_circuit

In [ ]:
partitions = [[(0,1),(1,3),(5,2)],[(0,(2,3)), (2,(4,3)), (3,(5,4))]]
p_phys = [0.6,0.3]
d = Depolar(n_ticks=6)
# d.generate(partitions, [[2],[1]], "SubsetSampler")
d.generate(partitions, p_phys, "DirectSampler")

[(0, 1), (5, 2), (3, (5, 4))]


0: {'Z': {1}}
1: {}
2: {}
3: {'X': {5}, 'Z': {4}}
4: {}
5: {'Y': {2}}

**TODO** Tests and example